# MadMiner particle physics tutorial

# Part 2a: Analyzing events at parton level

Johann Brehmer, Felix Kling, Irina Espejo, and Kyle Cranmer 2018-2019

In this second part of the tutorial, we'll generate events and extract the observables and weights from them. You have two options: In this notebook we'll do this at parton level, in the alternative part 2b we use Delphes.

## 0. Preparations

Before you execute this notebook, make sure you have a running installation of MadGraph.

In [16]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

from madminer.core import MadMiner
from madminer.lhe import LHEReader
from madminer.sampling import combine_and_shuffle
from madminer.plotting import plot_distributions


In [17]:
# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.DEBUG
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

Please enter here the path to your MG5 root directory:

In [18]:
mg_dir = '/home/llr/cms/cortinovis/EFT2Obs/MG5_aMC_v2_6_7'

## 1. Generate events

Let's load our setup:

In [19]:
miner = MadMiner()
miner.load("data/setup.h5")

22:04 madminer.utils.inter DEBUG   HDF5 file does not contain is_reference field.
22:04 madminer.core        INFO    Found 2 parameters:
22:04 madminer.core        INFO       CWL2 (LHA: dim6 2, maximal power in squared ME: (2,), range: (-20.0, 20.0))
22:04 madminer.core        INFO       CPWL2 (LHA: dim6 5, maximal power in squared ME: (2,), range: (-20.0, 20.0))
22:04 madminer.core        INFO    Found 6 benchmarks:
22:04 madminer.core        INFO       sm: CWL2 = 0.00e+00, CPWL2 = 0.00e+00
22:04 madminer.core        INFO       w: CWL2 = 15.20, CPWL2 = 0.10
22:04 madminer.core        INFO       neg_w: CWL2 = -1.54e+01, CPWL2 = 0.20
22:04 madminer.core        INFO       ww: CWL2 = 0.30, CPWL2 = 15.10
22:04 madminer.core        INFO       neg_ww: CWL2 = 0.40, CPWL2 = -1.53e+01
22:04 madminer.core        INFO       morphing_basis_vector_5: CWL2 = 17.62, CPWL2 = 19.63
22:04 madminer.core        INFO    Found morphing setup with 6 components
22:04 madminer.core        INFO    Did not find 

In a next step, MadMiner starts MadGraph to generate events and calculate the weights. You can use `run()` or `run_multiple()`; the latter allows to generate different runs with different run cards and optimizing the phase space for different benchmark points. 

In either case, you have to provide paths to the process card, run card, param card (the entries corresponding to the parameters of interest will be automatically adapted), and an empty reweight card. Log files in the `log_directory` folder collect the MadGraph output and are important for debugging.

The `sample_benchmark` (or in the case of `run_all`, `sample_benchmarks`) option can be used to specify which benchmark should be used for sampling, i.e. for which benchmark point the phase space is optimized. If you just use one benchmark, reweighting to far-away points in parameter space can lead to large event weights and thus large statistical fluctuations. It is therefore often a good idea to combine a lot of events at the "reference hypothesis" (for us the SM) and smaller samples from other benchmarks that span the parameter space.

One slight annoyance is that MadGraph only supports Python 2. The `run()` and `run_multiple()` commands have a keyword `initial_command` that let you load a virtual environment in which `python` maps to Python 2 (which is what we do below). Alternatively / additionally you can set `python2_override=True`, which calls `python2.7` instead of `python` to start MadGraph.

In [20]:
miner.run(
    sample_benchmark='sm',
    temp_directory = './',
    mg_directory=mg_dir,
    mg_process_directory='./mg_processes/signal1',
    proc_card_file='cards/proc_card_signal_hzz4l.dat',
    param_card_template_file='cards/param_card_template.dat',
    run_card_file='cards/run_card_signal_small.dat',
    log_directory='logs/signal',
    initial_command="PYTHONPATH=/usr/lib64/python",
    python2_override=True,
)

22:04 madminer.utils.inter INFO    Generating MadGraph process folder from cards/proc_card_signal_hzz4l.dat at ./mg_processes/signal1
22:10 madminer.core        INFO    Run 0
22:10 madminer.core        INFO      Sampling from benchmark: sm
22:10 madminer.core        INFO      Original run card:       cards/run_card_signal_small.dat
22:10 madminer.core        INFO      Original Pythia8 card:   None
22:10 madminer.core        INFO      Original config card:    None
22:10 madminer.core        INFO      Copied run card:         /madminer/cards/run_card_0.dat
22:10 madminer.core        INFO      Copied Pythia8 card:     None
22:10 madminer.core        INFO      Copied config card:      None
22:10 madminer.core        INFO      Param card:              /madminer/cards/param_card_0.dat
22:10 madminer.core        INFO      Reweight card:           /madminer/cards/reweight_card_0.dat
22:10 madminer.core        INFO      Log file:                run_0.log
22:10 madminer.core        INFO    Creat

In [21]:
additional_benchmarks = ['w', 'ww', 'neg_w', 'neg_ww']

In [23]:
miner.run_multiple(
    sample_benchmarks=additional_benchmarks,
    temp_directory = './',
    mg_directory=mg_dir,
    mg_process_directory='./mg_processes/signal2',
    proc_card_file='cards/proc_card_signal_hzz4l.dat',
    param_card_template_file='cards/param_card_template.dat',
    run_card_files=['cards/run_card_signal_small.dat'],
    log_directory='logs/signal',
    initial_command="PYTHONPATH=/usr/lib64/python",
    python2_override=True,
)

00:16 madminer.utils.inter INFO    Generating MadGraph process folder from cards/proc_card_signal_hzz4l.dat at ./mg_processes/signal2


KeyboardInterrupt: 

This will take a moment -- time for a coffee break!

After running any event generation through MadMiner, you should check whether the run succeeded: are the usual output files there, do the log files show any error messages? MadMiner does not (yet) perform any explicit checks, and if something went wrong in the event generation, it will only notice later when trying to load the event files.

### Backgrounds

We can also easily add other processes like backgrounds. An important option is the `is_background` keyword, which should be used for processes that do *not* depend on the parameters theta. `is_background=True` will disable the reweighting and re-use the same weights for all cross sections.

To reduce the runtime of the notebook, the background part is commented out here. Feel free to activate it and let it run during a lunch break.

In [ ]:
""""
miner.run(
    is_background=True,
    sample_benchmark='sm',
    temp_directory = './',
    mg_directory=mg_dir,
    mg_process_directory='./mg_processes/background',
    proc_card_file='cards/proc_card_background.dat',
    param_card_template_file='cards/param_card_template.dat',
    run_card_file='cards/run_card_background.dat',
    log_directory='logs/background',
    initial_command="PYTHONPATH=/usr/lib64/python",
    python2_override=True,
)
"""

Finally, note that both `MadMiner.run()` and `MadMiner.run_multiple()` have a `only_create_script` keyword. If that is set to True, MadMiner will not start the event generation directly, but prepare folders with all the right settings and ready-to-run bash scripts. This might make it much easier to generate Events on a high-performance computing system. 

## 2. Prepare analysis of the LHE samples

The `madminer.lhe` submodule allows us to extract observables directly from the parton-level LHE samples, including an approximate description of the detector response with smearing functions. The central object is an instance of the `LHEProcessor` class, which has to be initialized with a MadMiner file:

In [ ]:
lhe = LHEReader('data/setup.h5')

After creating the `LHEReader` object, one can add a number of event samples (the output of running MadGraph in step 1) with the `add_sample()` function.

In addition, you have to provide the information which sample was generated from which benchmark with the `sampled_from_benchmark` keyword, and set `is_background=True` for all background samples.

In [ ]:
lhe.add_sample(
    lhe_filename='mg_processes/signal1/Events/run_01/unweighted_events.lhe.gz',
    sampled_from_benchmark='sm',
    is_background=False,
    k_factor=1.,
)
for i, benchmark in enumerate(additional_benchmarks):
    lhe.add_sample(
        lhe_filename='mg_processes/signal2/Events/run_0{}/unweighted_events.lhe.gz'.format(i+1),
        sampled_from_benchmark=benchmark,
        is_background=False,
        k_factor=1.,
    )

"""
lhe.add_sample(
    lhe_filename='mg_processes/background/Events/run_01/unweighted_events.lhe.gz',
    sampled_from_benchmark='sm',
    is_background=True,
    k_factor=1.0,
"""


## 3. Smearing functions to model the detector response

Now we have to define the smearing functions that are used (in lieu of a proper shower and detector simulation). Here we will assume a simple 10% uncertainty on the jet energy measurements and a $\pm 0.1$ smearing for jet $\eta$ and $\phi$. The transverse momenta of the jets are then derived from the smeared energy and the on-shell condition for the quarks (this is what `pt_resolution_abs=None` does). The photons from the Higgs are assumed to be measured perfectly (otherwise we'd have to call `set_smearing` another time with `pdgis=[22]`).

In [ ]:
lhe.set_smearing(
    pdgids=[1,2,3,4,5,6,9,21,-1,-2,-3,-4,-5,-6],   # Partons giving rise to jets
    energy_resolution_abs=0.,
    energy_resolution_rel=0.1,
    pt_resolution_abs=None,
    pt_resolution_rel=None,
    eta_resolution_abs=0.1,
    eta_resolution_rel=0.,
    phi_resolution_abs=0.1,
    phi_resolution_rel=0.,
)

In addition, we can define noise that only affects MET. This adds Gaussian noise with mean 0 and std `abs_ + rel * HT` to MET_x and MET_y separately.

In [ ]:
lhe.set_met_noise(abs_=10., rel=0.05)

## 4. Observables and cuts

The next step is the definition of observables, either through a Python function or an expression that can be evaluated. Here we demonstrate the latter, which is implemented in `add_observable()`. In the expression string, you can use the terms `j[i]`, `e[i]`, `mu[i]`, `a[i]`, `met`, where the indices `i` refer to a ordering by the transverse momentum. In addition, you can use `p[i]`, which denotes the `i`-th particle in the order given in the LHE sample (which is the order in which the final-state particles where defined in MadGraph).

All of these represent objects inheriting from scikit-hep [LorentzVectors](http://scikit-hep.org/api/math.html#vector-classes), see the link for a documentation of their properties. In addition, they have `charge` and `pdg_id` properties.

`add_observable()` has an optional keyword `required`. If `required=True`, we will only keep events where the observable can be parsed, i.e. all involved particles have been detected. If `required=False`, un-parseable observables will be filled with the value of another keyword `default`.

In a realistic project, you would want to add a large number of observables that capture all information in your events. Here we will just define two observables, the transverse momentum of the leading (= higher-pT) jet, and the azimuthal angle between the two leading jets.

In [ ]:
lhe.add_observable(
    'pt_j1',
    'j[0].pt',
    required=False,
    default=0.,
)
lhe.add_observable(
    'delta_phi_jj',
    'j[0].deltaphi(j[1]) * (-1. + 2.*float(j[0].eta > j[1].eta))',
    required=True,
)
lhe.add_observable(
    'met',
    'met.pt',
    required=True,
)

We can also add cuts, again in parse-able strings. In addition to the objects discussed above, they can contain the observables:

In [ ]:
lhe.add_cut('(a[0] + a[1]).m > 122.')
lhe.add_cut('(a[0] + a[1]).m < 128.')
lhe.add_cut('pt_j1 > 20.')

## 5. Run analysis and store processes events

The function `analyse_samples` then calculates all observables from the LHE file(s) generated before, applies the smearing, and checks which events pass the cuts:

In [ ]:
lhe.analyse_samples()

The values of the observables and the weights are then saved in the HDF5 file. It is possible to overwrite the same file, or to leave the original file intact and save all the data into a new file as follows:

In [ ]:
lhe.save('data/lhe_data.h5')

## 6. Plot distributions

Let's see what our MC run produced:

In [ ]:
_ = plot_distributions(
    filename='data/lhe_data.h5',
    parameter_points=['sm', np.array([10.,0.])],
    line_labels=['SM', 'BSM'],
    uncertainties='none',
    n_bins=20,
    n_cols=3,
    normalize=True,
    sample_only_from_closest_benchmark=True
)

## 7. Combine and shuffle different samples

To reduce disk usage, you can generate several small event samples with the steps given above, and combine them now. Note that (for now) it is essential that all of them are generated with the same setup, including the same benchmark points / morphing basis!

This is generally good practice even if you use just one sample, since the events might have some inherent ordering (e.g. from sampling from different hypotheses). Later when we split the events into a training and test fraction, such an ordering could cause problems.

In [ ]:
combine_and_shuffle(
    ['data/lhe_data.h5'],
    'data/lhe_data_shuffled.h5'
)

In [1]:
import h5py

def get_all(name):
    print(name)

with h5py.File('data/lhe_data.h5', 'r') as f:
    g_name = f.visit(get_all)
    d = f['samples/weights']
    print(d[:])

benchmarks
benchmarks/is_nuisance
benchmarks/is_reference
benchmarks/names
benchmarks/values
morphing
morphing/components
morphing/morphing_matrix
observables
observables/definitions
observables/names
parameters
parameters/lha_blocks
parameters/lha_ids
parameters/max_power
parameters/names
parameters/ranges
parameters/transforms
sample_summary
sample_summary/background_events
sample_summary/signal_events_per_benchmark
samples
samples/observations
samples/sampling_benchmarks
samples/weights
[[2.0726000e-08 2.7529983e-08 2.0352479e-07 2.5456669e-08 1.9068228e-07
  6.1808753e-07]
 [2.0726000e-08 3.9610023e-09 6.7001026e-08 2.6260770e-07 5.6471020e-08
  1.3612787e-08]
 [2.0726000e-08 2.5217389e-08 1.8276828e-08 3.2847775e-08 1.0833700e-07
  1.1893883e-07]
 ...
 [1.5961384e-09 4.1588000e-06 4.4923203e-06 1.9897541e-06 2.0056861e-06
  1.3868649e-05]
 [2.0726000e-08 2.2278555e-08 2.8757380e-08 6.5551634e-08 2.3185626e-07
  2.9023953e-07]
 [5.4585992e-09 4.1267936e-07 6.9135251e-07 4.9158000e-